In [0]:
filename = dbutils.widgets.get('filename')
Fnamewithoutext = filename.split('.')[0]
print(filename)

In [0]:
# Checking all our mount points
dbutils.fs.mounts()

Out[19]: [MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/Volumes', source='UnityCatalogVolumes', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType=''),
 MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType=''),
 MountInfo(mountPoint='/Volume', source='DbfsReserved', encryptionType=''),
 MountInfo(mountPoint='/mnt/sales', source='wasbs://sales@cdstorageaccount1.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/volumes', source='DbfsReserved', encryptionType=''),
 MountInfo(mountPoint='/', source='DatabricksRoot', encryptionType=''),
 MountInfo(mountPoint='/volume', source='DbfsReserved', encryptionType='')]

In [0]:
# Check if storage account is mounted
mounted = False
for x in dbutils.fs.mounts():
    if x.mountPoint == '/mnt/sales':
        mounted = True
        break
    else:
        mounted = False


print(mounted)

False

In [0]:
if mounted == False:
    extra_configs = {
    'fs.azure.account.key.cdstorageaccount1.blob.core.windows.net': dbutils.secrets.get('cd-secrets-databricks-scope', 'storagekey')}

    dbutils.fs.mount(
        source='wasbs://sales@cdstorageaccount1.blob.core.windows.net',
        mount_point='/mnt/sales',
        extra_configs=extra_configs
    )


In [0]:
orders_df = spark.read.csv(f'dbfs:/mnt/sales/Landing/{filename}',inferSchema=True,header=True)

In [0]:
orders_df.show()

+--------+-------------------+-----------+---------------+
order_id| order_date|customer_id| order_status|
+--------+-------------------+-----------+---------------+
 1111111|2013-07-25 00:00:00| 11599| CLOSED|
 2222222|2013-07-25 00:00:00| 256|PENDING_PAYMENT|
 3333333|2013-07-25 00:00:00| 12111| COMPLETE|
 4444444|2013-07-25 00:00:00| 8827| CLOSED|
+--------+-------------------+-----------+---------------+

### Checking for condition 1 i.e. duplicate order id's, if yes then move the file to discarded folder and exit notebook with a message or else create a temporary table

In [0]:
errorflg = False
orders_count = orders_df.count()
dist_orders = orders_df.select('order_id').distinct().count()
if orders_count!=dist_orders:
    errorflg = True
if errorflg == True:
    dbutils.fs.mv(f'/mnt/sales/Landing/{filename}','/mnt/sales/Discarded')
    dbutils.notebook.exit('{"errorflg":"True","errorMsg":"Orderid is repeated"}')




### Then the second validation(check for valid orders_status) is evaluated.This will be compared with the list of all unique order statuses we saved in azure sql database

In [0]:
jdbcHostname = "cd-server-24.database.windows.net"
jdbcPort = 1433
jdbcDatabase = "cd_database_24"
jdbcTable = "dbo.valid_order_status"
jdbcUsername = "cdadmin"
jdbcPassword = dbutils.secrets.get('cd-secrets-databricks-scope', 'sql-passw')
jdbcDriver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

jdbcUrl = f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};database={jdbcDatabase};user ={jdbcUsername};password={jdbcPassword}"


In [0]:
df_status = spark.read.format("jdbc").option("url",jdbcUrl).option("dbtable",jdbcTable).load()

In [0]:
display(df_status)

status_name
ON_HOLD
PAYMENT_REVIEW
PROCESSING
CLOSED
SUSPECTED_FRAUD
COMPLETE
PENDING
CANCELED
PENDING_PAYMENT


In [0]:
values_to_check = [df_status.collect()[i][0] for i in range (0,df_status.count())]
filtered_df = orders_df.filter(~orders_df.order_status.isin(values_to_check)).select('order_status')
filtered_df.show()


+------------+
order_status|
+------------+
+------------+

In [0]:
filtered_df.count()

Out[115]: 0

In [0]:
if filtered_df.count()>0:
    errorflg = True

if errorflg == True:
    dbutils.fs.mv(f'/mnt/sales/Landing/{filename}','/mnt/sales/Discarded')
    dbutils.notebook.exit('{"errorflg":"True","errorMsg":"Invalid order status found"}')
else:
    dbutils.fs.mv(f'/mnt/sales/Landing/{filename}','/mnt/sales/Staging')
    dbutils.notebook.exit('{"errorflg":"False","errorMsg":"All Good"}')
